In [7]:
import time
import pandas as pd
import os
import pygame
import sys

import pygame.mixer  # Importar el módulo de sonido





# Clase que toma las medidas de la pantalla para ubicar las imágenes
class ScreenRect:
    origin = 0, 0
    scale = 1.0

    @classmethod
    def setGeometry(cls):
        screenRect = pygame.display.get_surface().get_rect()
        W, H = screenRect.size
        cls.scale = min(W, H)
        cls.origin = (W - cls.scale) / 2, (H - cls.scale) / 2

# Clase para mostrar palabras
class StimWord:
    pygame.font.init()
    font = pygame.font.Font(None, 70)
    color = pygame.Color('white')
    
    def __init__(self, word, max_width, max_height):
        self.word = word
        self.max_width = max_width
        self.max_height = max_height
        self.lines = self.wrap_text(word)
        self.images = [StimWord.font.render(line, True, StimWord.color) for line in self.lines]
        self.rects = [image.get_rect() for image in self.images]
        self.position_rects()

    def wrap_text(self, text):
        words = text.split(' ')
        lines = []
        current_line = []
        for word in words:
            current_line.append(word)
            line_width, _ = StimWord.font.size(' '.join(current_line))
            if line_width > self.max_width:
                current_line.pop()
                lines.append(' '.join(current_line))
                current_line = [word]
        lines.append(' '.join(current_line))
        return lines

    def position_rects(self):
        total_height = sum(rect.height for rect in self.rects)
        screen_rect = pygame.display.get_surface().get_rect()
        start_y = (screen_rect.height - total_height) // 2
        for rect in self.rects:
            rect.centerx = screen_rect.centerx
            rect.y = start_y
            start_y += rect.height

    def display(self):
        screen = pygame.display.get_surface()
        for image, rect in zip(self.images, self.rects):
            screen.blit(image, rect)
        pygame.display.update()


# Función para leer el archivo Excel
def leer_excel(ruta):
    df = pd.read_excel(ruta)
    contextos = df['Contexto'].tolist()
    frases1 = df['Frase_1'].tolist()
    frases2 = df['Frase_2'].tolist()
    audios1 = df['Audios'].tolist()  # Nueva columna para los audios
    return contextos, frases1, frases2, audios1


# Clase principal ERT
class ERT:
    @staticmethod
    def run():
        # Inicializar Pygame
        pygame.init()
        pygame.mixer.init()
        pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        pygame.mouse.set_visible(False)
        ScreenRect.setGeometry()

        # Leer los contextos y frases desde el archivo Excel
        contextos, frases1, frases2, audios1 = leer_excel('./Pruebas.xlsx')

        while True:
            for contexto, frase1, frase2, audio1 in zip(contextos, frases1, frases2, audios1):
                # Mostrar el contexto
                pygame.display.get_surface().fill((0, 0, 0))
                contexto_display = StimWord(contexto, 800, 600)
                contexto_display.display()

                # Reproducir el audio y esperar a que se presione la tecla espacio
                pygame.mixer.music.load(audio1)  # Cargar el archivo de audio
                pygame.mixer.music.play()  # Reproducir el audio

                # Esperar a que se presione la tecla espacio
                esperando = True
                while esperando:
                    for evento in pygame.event.get():
                        if evento.type == pygame.KEYDOWN:
                            if evento.key == pygame.K_SPACE:
                                esperando = False
                            elif evento.key == pygame.K_ESCAPE:
                                pygame.quit()
                                sys.exit()

                

                # pygame.display.get_surface().fill((0, 0, 0))
                # frase1_display = StimWord(frase1, 800, 600)
                # frase1_display.display()

                # esperando_audio = True
                # while esperando_audio:
                #     for evento in pygame.event.get():
                #         if evento.type == pygame.KEYDOWN:
                #             if evento.key == pygame.K_SPACE:
                #                 pygame.mixer.music.stop()  # Detener el audio
                #                 esperando_audio = False
                #             elif evento.key == pygame.K_ESCAPE:
                #                 pygame.quit()
                #                 sys.exit()

                # Mostrar las otras frases por los tiempos especificados
    
                for frase, tiempo in [(frase1, 1000), (frase2, 1000)]:
                    pygame.display.get_surface().fill((0, 0, 0))
                    frase_display = StimWord(frase, 800, 600)
                    frase_display.display()
                    pygame.time.wait(tiempo)

                # Volver a mostrar el contexto
                pygame.display.get_surface().fill((0, 0, 0))
                contexto_display.display()


if __name__ == '__main__':
    ERT.run()


Frase 1: Lucía le espera a la salida de clase y le da un 
Frase 2: Cachetada
Frase 1: A Federico le llama el jefe de estudios y le 
Frase 2: Perro
Frase 1: La empresa de Carla entra en 
Frase 2: Gato
Frase 1: La última vez que fuimos a un pub que él sugirió, lo pasamos
Frase 2: Ladro


KeyboardInterrupt: 

: 

In [2]:
import pandas as pd

# Ruta del archivo de Excel
file_path = './4condiciones_160escenarios_EVA.xlsx'

# Cargar el archivo de Excel
xls = pd.ExcelFile(file_path)

# Crear una variable para cada hoja y extraer solo las columnas 3 y 4
sheet1 = xls.parse(xls.sheet_names[0]).iloc[:, [8, 2, 3, 4]]
sheet2 = xls.parse(xls.sheet_names[1]).iloc[:, [8, 2, 3, 4]]
sheet3 = xls.parse(xls.sheet_names[2]).iloc[:, [8, 2, 3, 4]]
sheet4 = xls.parse(xls.sheet_names[3]).iloc[:, [8, 2, 3, 4]]

# Crear listas para cada una de las cuatro condiciones
grupo1 = []
grupo2 = []
grupo3 = []
grupo4 = []

# Número de elementos por condición
num_elements = 160

# Crear las listas combinadas 
for i in range(0, num_elements, 4):
    grupo1.extend([
        sheet1.iloc[i % len(sheet1)].tolist(),
        sheet2.iloc[(i+1) % len(sheet2)].tolist(),
        sheet3.iloc[(i+2) % len(sheet3)].tolist(),
        sheet4.iloc[(i+3) % len(sheet4)].tolist()
    ])
    grupo2.extend([
        sheet1.iloc[(i+1) % len(sheet1)].tolist(),
        sheet2.iloc[(i+2) % len(sheet2)].tolist(),
        sheet3.iloc[(i+3) % len(sheet3)].tolist(),
        sheet4.iloc[(i+4) % len(sheet4)].tolist()
    ])
    grupo3.extend([
        sheet1.iloc[(i+2) % len(sheet1)].tolist(),
        sheet2.iloc[(i+3) % len(sheet2)].tolist(),
        sheet3.iloc[(i+4) % len(sheet3)].tolist(),
        sheet4.iloc[(i+5) % len(sheet4)].tolist()
    ])
    grupo4.extend([
        sheet1.iloc[(i+3) % len(sheet1)].tolist(),
        sheet2.iloc[(i+4) % len(sheet2)].tolist(),
        sheet3.iloc[(i+5) % len(sheet3)].tolist(),
        sheet4.iloc[(i+6) % len(sheet4)].tolist()
    ])


In [3]:
grupo1

[['Social  Alineada',
  'Carlos es un chico dulce y sensible enamorado de una chica de su clase llamada Lucía. Es muy atento con ella y le recuerda lo guapa que va a clase todos los días. Un buen día, se arma de valor y la invita a salir.',
  'Lucía le espera a la salida de clase y le da un ',
  'beso.'],
 ['Social  Desalineada',
  'Federico es el más listo de su clase. Siempre ayuda a sus compañeros con los deberes. Sobre todo, se preocupa por Roberto, un chico al que le van mal los estudios. Roberto está dejando de ir al colegio.',
  'A Federico le llama el jefe de estudios y le ',
  'expulsa.'],
 ['Antisocial Alineada',
  'Carla es directora de una empresa con un futuro brillante. Carla comienza a saborear las mieles del éxito y se vuelve inflexible y dura con sus empleados. Los carga de trabajo y reduce su sueldo. En algunos casos, los despide sin previo aviso.',
  'La empresa de Carla entra en ',
  'bancarrota.'],
 ['Antisocial Desalineada',
  'Javier es muy mandón. Siempre va de 